Sure! Let's break down the entire code snippet in detail. This code appears to be part of a web application that implements user authentication using password hashing and JSON Web Tokens (JWTs) for session management.

### Logger Setup

```python
logger = logging.getLogger(__name__)
```

- **Purpose**: Initializes a logger specific to this module.
- **`__name__`**: Represents the name of the current module, allowing you to identify where logs are coming from in a larger application. This is useful for debugging and tracking application behavior.

### Password Encryption Context

```python
bcrypt_context = CryptContext(schemes=["pbkdf2_sha256", "bcrypt"], deprecated="auto")
```

- **Purpose**: Sets up a context for password hashing.
- **`CryptContext`**: A class from the `passlib` library that manages various password hashing schemes.
- **`schemes`**: Specifies the hashing algorithms to be used. In this case, `pbkdf2_sha256` and `bcrypt` are included, which are both secure options for hashing passwords.
- **`deprecated="auto"`**: Automatically manages deprecated schemes. If you use an algorithm that is considered outdated, it will alert you.

### OAuth2 Password Bearer Scheme

```python
oauth2_bearer = OAuth2PasswordBearer(tokenUrl='/auth/token/json')
```

- **Purpose**: Initializes the OAuth2 password bearer authentication scheme.
- **`OAuth2PasswordBearer`**: A class from FastAPI that helps manage token-based authentication.
- **`tokenUrl`**: Specifies the endpoint where users can send their credentials to receive a JWT.

### Dictionary for Logged-in Users

```python
logged_in_users = {}
```

- **Purpose**: A simple in-memory dictionary to keep track of currently logged-in users and their tokens.
- **Structure**: Keys are user identifiers (like user IDs), and values are the corresponding JWT tokens. This allows for quick validation of tokens.

### User Authentication Function

```python
def authenticate_user(email: str, password: str, session: Session):
```

- **Purpose**: Authenticates a user based on their email and password.
- **Parameters**:
  - `email`: The user's email address.
  - `password`: The plaintext password provided during login.
  - `session`: A database session for querying user data.

#### Inside the Function

```python
user = session.query(User).filter(User.email == email).first()
```

- **Database Query**: Attempts to find the user in the database by their email.
- **`User`**: Presumably a model that represents the user table in your database.

```python
if not user or not bcrypt_context.verify(password, user.hashed_password):
    raise HTTPException(status_code=401, detail="Invalid email or password")
```

- **Verification**: Checks if the user exists and if the provided password matches the stored hashed password.
- **`HTTPException`**: Raises an exception with a 401 status code if the credentials are invalid.

```python
return user
```

- **Return Value**: If the user is authenticated successfully, the user object is returned.

### Create Access Token Function

```python
def create_access_token(data: Dict[str, str], expires_delta: timedelta = None):
```

- **Purpose**: Creates a JWT access token with user data and an optional expiration time.
- **Parameters**:
  - `data`: User data to encode in the token.
  - `expires_delta`: Optional time duration for token expiration.

#### Inside the Function

```python
to_encode = data.copy()
```

- **Data Copy**: Creates a mutable copy of the data to avoid modifying the original dictionary.

```python
if expires_delta:
    to_encode.update({"exp": datetime.now(timezone.utc) + expires_delta})
else:
    to_encode.update({"exp": datetime.now(timezone.utc) + timedelta(minutes=15)})
```

- **Expiration Time**: Sets the expiration time for the token, either based on the provided `expires_delta` or defaults to 15 minutes.

```python
return jwt.encode(to_encode, SECRET_KEY, algorithm=ALGORITHM)
```

- **JWT Encoding**: Encodes the data into a JWT using the specified secret key and algorithm, and returns the token.

### Get Current User Function

```python
def get_current_user(token: str = Depends(oauth2_bearer), session: Session = Depends(get_db)):
```

- **Purpose**: Retrieves the currently logged-in user from the provided JWT token.
- **Parameters**:
  - `token`: The JWT token sent in the request.
  - `session`: A database session for querying user data.

#### Inside the Function

```python
try:
    payload = jwt.decode(token, SECRET_KEY, algorithms=[ALGORITHM])
```

- **Token Decoding**: Attempts to decode the JWT using the secret key and specified algorithm to extract the payload.

```python
user_id: str = payload.get("id")
```

- **User ID Extraction**: Gets the user ID from the decoded payload.

```python
if user_id is None or user_id not in logged_in_users or logged_in_users[user_id] != token:
    raise HTTPException(status_code=401, detail="Could not validate credentials")
```

- **Validation**: Checks if the user ID is valid and matches the token stored in `logged_in_users`. Raises an exception if validation fails.

```python
user = session.query(User).filter(User.id == user_id).first()
```

- **Database Query**: Fetches the user object from the database using the user ID.

```python
if user is None:
    raise HTTPException(status_code=401, detail="Could not validate credentials")
```

- **Final Validation**: Ensures that the user exists in the database.

```python
return user
```

- **Return Value**: Returns the authenticated user object.

### Retrieve All Logged-in Users Function

```python
def get_logged_in_users(session: Session, token: str):
```

- **Purpose**: Retrieves details of all users currently logged in.
- **Parameters**:
  - `session`: A database session for querying users.
  - `token`: The JWT token of the current user.

#### Inside the Function

```python
try:
    payload = jwt.decode(token, SECRET_KEY, algorithms=[ALGORITHM])
```

- **Token Decoding**: Same as in `get_current_user`, retrieves the payload from the JWT.

```python
current_user_id: str = payload.get("id")
```

- **User ID Extraction**: Retrieves the ID of the current user.

```python
if not current_user_id:
    raise HTTPException(status_code=401, detail="Could not validate credentials")
```

- **Validation**: Checks if the user ID is valid.

```python
if current_user_id not in logged_in_users:
    raise HTTPException(status_code=403, detail="Not authorized to view logged-in users")
```

- **Authorization Check**: Ensures that the current user is authorized to view the list of logged-in users.

```python
logged_in_user_ids = list(logged_in_users.keys())
users_details = []
```

- **User Retrieval**: Prepares to collect details of all logged-in users.

```python
for user_id in logged_in_user_ids:
    user = session.query(User).filter(User.id == user_id).first()
    if user:
        user_details = UserDetails(
            email=user.email,
            user_name=user.user_name
        )
        users_details.append(user_details)
```

- **Loop Through Logged-in Users**: Queries the database for each logged-in user and gathers their details (like email and username) into a list.

```python
return users_details
```

- **Return Value**: Returns a list of `UserDetails` objects representing all logged-in users.

### Exception Handling

The functions include exception handling to manage potential errors:

- **HTTPException**: Used to return error responses with appropriate HTTP status codes.
- **General Exception**: Catches any unexpected errors during execution.

### Summary

- The code establishes a logging mechanism, sets up password encryption, and implements user authentication using JWTs.
- It includes functions for user authentication, token creation, retrieving the current user from a token, and listing all logged-in users.
- This structure helps maintain secure user sessions and manage access control effectively.

If you have any specific questions or need further clarification on any part, feel free to ask!